# 🔩 에이전트에 미들웨어 추가하기

 에이전트에 미들웨어를 추가하여 로깅, 보안 및 기타 공통 관심사를 위해 에이전트 상호 작용을 가로채고 수정하는 방법을 알아봅니다. 미들웨어에 대한 내용은 수시로 업데이트가 이루어지고 있으므로, 최신 정보를 확인하는 것을 권장드립니다. 
 
 ✨ [미들웨어 문서](https://learn.microsoft.com/en-us/agent-framework/user-guide/agents/agent-middleware?pivots=programming-language-python)를 참조하세요.   
 특히, 미들웨어 예외 처리, 스코프, 터미네이션과 가드레일 등 고급적인 내용에 대해서는 공식 문서들을 참고하시기 바랍니다.

---

## ✍️ 미들웨어 작성하기

Agent Framework의 미들웨어는 실행의 다양한 단계에서 에이전트 상호 작용을 **가로채고, 수정하고, 개선**할 수 있는 강력한 방법을 제공합니다. 미들웨어를 사용하면 핵심 에이전트 또는 함수 로직을 수정하지 않고도 로깅, 보안 유효성 검사, 오류 처리 및 결과 변환과 같은 **cross-cutting 관심사**를 구현할 수 있습니다.

에이전트 프레임워크는 다음 **세 가지 유형의 미들웨어**를 지원합니다:

| 유형 | 설명 |
|------|------|
| 🤖 **에이전트 미들웨어** | 에이전트 실행을 가로채어 입력, 출력 및 제어 흐름을 검사하고 수정합니다 |
| 🔧 **함수 미들웨어** | 에이전트 실행 중 함수(도구) 호출을 가로채어 입력 유효성 검사, 결과 변환 및 실행 제어를 가능하게 합니다 |
| 💬 **채팅 미들웨어** | AI 모델로 전송되는 기본 채팅 요청을 가로채어 원시 메시지, 옵션 및 응답에 대한 접근을 제공합니다 |



### 📌 Agent 미들웨어

에이전트 기반 미들웨어는 에이전트 실행을 가로채고 수정합니다.   
`AgentContext `를 사용하며, 다음 속성을 제공합니다:

| 속성 | 설명 |
|------|------|
| **agent** | 호출되는 에이전트 |
| **messages** | 대화 내 채팅 메시지 목록 |
| **is_streaming** | 응답이 스트리밍 중인지를 나타내는 Boolean 값 |
| **metadata** | 미들웨어 간에 추가 데이터를 저장하기 위한 딕셔너리 |
| **result** | 에이전트의 응답 (수정 가능) |
| **terminate** | 추가 처리를 중지하는 플래그 |
| **kwargs** | 에이전트 run 메서드에 전달되는 추가 키워드 인수 |

> **참고**
>
> 코드 중 `next` 호출은 미들웨어 체인을 이어가며, 현재가 마지막 미들웨어라면 에이전트를 실행합니다.

In [ ]:
# Function-based Middleware 예시

from agent_framework import AgentContext
from typing import Callable, Awaitable

async def logging_agent_middleware(
    context: AgentContext,
    next: Callable[[], Awaitable[None]],
) -> None:
    """ 에이전트 실행정보를 로깅하는 간단한 미들웨어 """

    print("[Agent] 실행 시작...")

    # 계속해서 에이전트를 실행
    await next()

    print("[Agent] 실행 종료!")


In [ ]:
# Class-based Middleware 예시
#------------------------------------------------------------
# 클래스 기반 미들웨어는 process 메서드를 사용하며, 
# 이 메서드는 함수 기반 미들웨어와 동일한 시그니처와 동작을 가집니다.
from agent_framework import AgentMiddleware, AgentContext
from typing import Callable, Awaitable

class LoggingAgentMiddleware(AgentMiddleware):
    """ 에이전트 실행정보를 로깅하는 간단한 미들웨어"""

    async def process(
        self,
        context: AgentContext,
        next: Callable[[], Awaitable[None]],
    ) -> None:
        print("[Agent 클래스] 실행 시작...")
        await next()
        print("[Agent 클래스] 실행 종료!")


---

## 🤖 Agent 생성 및 미들웨어 추가

In [ ]:
from agent_framework.azure import AzureOpenAIChatClient
from azure.identity import AzureCliCredential

agent = AzureOpenAIChatClient(credential=AzureCliCredential()).as_agent(
    name="GreetingAgent",
    instructions="당신은 한국어로 답변하는, 도움이 되는 어시스턴트입니다. 🤖✨",
    middleware=[LoggingAgentMiddleware()]   # 여기에 미들웨어 추가
    # middleware=[logging_agent_middleware()]   
)

async def main():
    result = await agent.run("안녕하세요!")
    print(result.text)

await main()

---

## 🔧 Function 기반 미들웨어

에이전트가 Function 도구를 사용하는 경우, Function 기반 미들웨어를 적용하면 함수 호출도 가로챌 수 있습니다.

### 📋 FunctionInvocationContext 속성

Function 기반 미들웨어는 에이전트 내의 함수 호출을 가로챕니다.   
`FunctionInvocationContext`를 사용하며, 다음 속성을 제공합니다:

| 속성 | 설명 |
|------|------|
| **function** | 호출되는 함수 |
| **arguments** | 함수에 대한 검증된 인수 |
| **metadata** | 미들웨어 간에 추가 데이터를 저장하기 위한 딕셔너리 |
| **result** | 함수의 반환 값 (수정 가능) |
| **terminate** | 추가 처리를 중지하는 플래그 |
| **kwargs** | 이 함수를 호출한 채팅 메서드에 전달되는 추가 키워드 인수 |  


In [ ]:
from agent_framework import FunctionInvocationContext

def get_time():
    """현재 시간을 가져옵니다."""
    from datetime import datetime
    return datetime.now().strftime("%H:%M:%S")

async def logging_function_middleware(
    context: FunctionInvocationContext,
    next: Callable[[], Awaitable[None]],
) -> None:
    """함수 호출을 기록하는 미들웨어입니다."""
    print(f"function 실행 시작 : {context.function.name}")

    await next()

    print(f"function 실행 완료: {context.result}")

# Add both the function and middleware to your agent
agent = AzureOpenAIChatClient(credential=AzureCliCredential()).as_agent(
    name="TimeAgent",
    instructions="현재 시간을 알려주세요.",
    tools=[get_time],
    middleware=[logging_function_middleware],
) 
async def main():
    result = await agent.run("현재 시간이 어떻게 되나요?")
    print(result.text)

await main()


---

## 🎯 Run(Chat) 수준에서 미들웨어 사용

Chat 미들웨어는 AI 모델에 전송되는 채팅 요청을 가로채는 역할을 합니다. 


In [ ]:
# Function-based Chat Middleware 예시

from agent_framework import ChatContext
from typing import Callable, Awaitable

async def logging_chat_middleware(
    context: ChatContext,
    next: Callable[[ChatContext], Awaitable[None]],
) -> None:
    """AI 인터렉션을 로깅하는 Chat 미들웨어"""
    # 전처리 : AI 호출 전에 로깅
    print(f"[Chat] {len(context.messages)}개의 메시지를 AI로 전송하는 중...")

    # 다음 미들웨어 혹은 AI 서비스로 넘김
    await next()

    # 후-처리 : AI 응답 후에 로깅
    print("[Chat] AI 응답 수신 완료")

In [ ]:
from agent_framework.azure import AzureOpenAIChatClient
from azure.identity import AzureCliCredential

agent = AzureOpenAIChatClient(credential=AzureCliCredential()).as_agent(
    name="TimeAgent",
    instructions="당신은 현재 시간을 알려주는 에이전트입니다."
) 

async def main():

    # Use middleware for this specific run only
    result = await agent.run(
        "좋은 하루 되세요!",
        middleware=[logging_chat_middleware]
    )

    print("Chat 미들웨어 있는 결과 : " + result.text)    
    print("=" * 70)
    result = await agent.run("무엇을 알려주실 수 있나요?")
    print("Chat 미들웨어 없는 결과 : " + result.text)

await main()

## 🤝 Agent 미들웨어 + Chat 미들웨어

Agent 미들웨어와 Chat 미들웨어는 함께 사용할 수 있으며, 다음과 같은 방식으로 적용됩니다:

| 구분 | 설명 |
|------|------|
| **Agent 수준** | 모든 실행에서 지속적으로 유지되며, 에이전트 생성 시 한 번 구성됩니다 |
| **Run 수준** | 특정 실행에서만 적용되며, 요청 별로 사용자 정의가 가능합니다 |
| **실행 순서** | Agent 미들웨어 (가장 바깥) → Run 미들웨어 (가장 안쪽) → 에이전트 실행 |


In [ ]:
agent = AzureOpenAIChatClient(credential=AzureCliCredential()).as_agent(
    name="TimeAgent",
    instructions="당신은 현재 시간을 알려주는 에이전트입니다.",
    middleware=[LoggingAgentMiddleware()] 
) 

async def main():

    # Use middleware for this specific run only
    result = await agent.run(
        "좋은 하루 되세요!",
        middleware=[logging_chat_middleware]
    )

    print("Chat 미들웨어 있는 결과 : " + result.text)    
    print("=" * 70)
    result = await agent.run("무엇을 알려주실 수 있나요?")
    print("Chat 미들웨어 없는 결과 : " + result.text)

await main()

## 🛑 미들웨어 종료

미들웨어는 context.terminate를 사용하여 조기에 실행을 종료할 수 있습니다.   
이는 보안 점검, rate 제한, 유효성 검증 실패와 같은 상황에서 유용합니다.

In [ ]:
# Function-based Blocking Middleware 예시

from agent_framework import AgentContext
from typing import Callable, Awaitable

async def blocking_middleware(
    context: AgentContext,
    next: Callable[[AgentContext], Awaitable[None]],
) -> None:
    """미들웨어는 조건에 기반하여 실행을 차단합니다. blocked이라는 글자가 있으면 요청을 차단합니다."""

    # 차단할 컨텐츠를 검사합니다.
    last_message = context.messages[-1] if context.messages else None
    if last_message and last_message.text:
        if "blocked" in last_message.text.lower():
            print("**** 미들웨어에 의해 요청이 차단되었습니다.****")
            context.terminate = True
            return

    # 문제가 없다면, 계속해서 에이전트를 실행합니다.
    await next(context)

In [ ]:
agent = AzureOpenAIChatClient(credential=AzureCliCredential()).as_agent(
    name="TimeAgent",
    instructions="당신은 보안을 매우 중요하게 생각하는 에이전트입니다."
) 

async def main():

    result = await agent.run(
        "이 메시지는 blocked 단어를 포함하고 있습니다. 고로 차단되어야 하는데...",
        middleware=[blocking_middleware]
    )

    print("결과 : " + (result.text if result else "요청이 차단되었습니다."))    
    print("=" * 70)
    
await main()
